In [1]:
# Connect to drive
from google.colab import drive
drive.mount("/content/drive")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install ghostscript
!pip install camelot-py[cv]
!pip install excalibur-py
!apt install ghostscript python3-tk
!pip install tika
!pip install pypdf
!pip install python-dotenv
!pip install 'PyPDF2<3.0'

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ghostscript is already the newest version (9.55.0~dfsg1-0ubuntu5.6).
python3-tk is already the newest version (3.10.8-1~22.04).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [3]:
import requests
import camelot
import json
import logging
import re
import pandas as pd
import numpy as np
import os
from tika import parser
from pypdf import PdfReader
import dotenv
dotenv.load_dotenv('.env')


True

In [4]:
DATA_FOLDER = os.environ.get('DATA_FOLDER')

In [5]:
# A check to view downloaded files
lei_files = os.listdir(DATA_FOLDER+'/Leading Economic Indicators')
lei_files = [i for i in lei_files if '.pdf' in i]


In [6]:
def get_table_page(lei_path):
  reader = PdfReader(lei_path)
  text = ""

  search_text1 = "TABLE 1:[\w\s]*KENYA CONSUMER PRICE INDICES"


  search_text3 = "TABLE 1(B): KENYA CONSUMER PRICE INDICES"
  count = 0
  x = []
  counts = []

  for page in reader.pages:
      page_extract = page.extract_text().upper().replace('  ',' ').replace('  ',' ')
      count+=1
      if len(x)==0:
        try:
            x1 = re.search(search_text1, page_extract)
            if x1 and count>3:
                return count
        except:
            x1 = re.search(search_text2, page_extract)
            if x1 and count>3:
                counts.append(count)

      else:
        x2 = re.search(search_text3, page_extract)
        if x2 and count>3:
            counts.append(count)
            return counts






In [7]:
def get_headers(lei_path, table_page):
  page = str(table_page)
  header_catch = 'OVERALL MONTH ON MONTH INFLATION'
  header_row = 0
  lattice_tables_list = camelot.read_pdf(lei_path, pages=page, flavor='lattice', strip_text='\n')

  if len(lattice_tables_list)>0:
    lattice_table = lattice_tables_list[0].df
    for idx, row in lattice_table.iterrows():
      headers_caught = [i for i in row if header_catch in i.upper()]
      if len(headers_caught)>0:
        header_row=idx

    headers_table = camelot.read_pdf(lei_path, pages=page, flavor='lattice', strip_text='\n')[0].df
    headers = list(headers_table.iloc[header_row])
    return headers
  else:
    return 'No lattice table'


In [8]:
def get_table_page_tmp(lei_path):
  reader = PdfReader(lei_path)
  search_text1a = "TABLE1:[\w]*CONSUMERPRICEINDICES"
  search_text1b = "UNDERLYING"
  search_text2 = "TABLE1\(A\):[KENYA]*CONSUMERPRICEINDICES"
  search_text3 = "TABLE1\(B\):[KENYA]*INFLATIONRATE"

  count = 0
  table_page_dict = {}
  counts = []

  for page in reader.pages:
      page_extract = page.extract_text().upper().replace(' ','')
      count+=1
      if len(table_page_dict)==0:
        cpi_inf_match = re.search(search_text1a, page_extract)
        inf_match = re.search(search_text1b, page_extract)
        if cpi_inf_match and inf_match and count>3:
          table_page_dict.update({'cpi_inf_match':count})
        else:
          cpi_match = re.search(search_text2, page_extract)
          if cpi_match and count>3:
            table_page_dict.update({'cpi_match':count})

      if 'cpi_match' in table_page_dict:
        inf_match = re.search(search_text3, page_extract)
        if inf_match and count>3:
            table_page_dict.update({'inf_match':count})
  return table_page_dict

In [9]:
cpi_inf_page_list = []
for file_no,lei_file in enumerate(lei_files):
  lei_path = DATA_FOLDER+'/Leading Economic Indicators/'+lei_file
  table_pages = get_table_page_tmp(lei_path)
  cpi_inf_page_list.append(table_pages)

In [45]:
def get_cpi_inf_data(lei_path, table_page):
  df = camelot.read_pdf(lei_path, pages=table_page, flavor='stream', strip_text=' \n')[0].df
  header_catch = 'OVERALL MONTH ON MONTH INFLATION'
  header_row = 1
  lattice_table = camelot.read_pdf(lei_path, pages=str(table_page), flavor='lattice', strip_text='\n')[0].df
  for idx, row in lattice_table.iterrows():
    for i in row:
      if header_catch in i.upper():
        header_row=idx

  headers = list(camelot.read_pdf(lei_path, pages=str(table_page), flavor='lattice', strip_text='\n')[0].df.iloc[header_row])
  headers = [i.replace('*','').replace('  ',' ').upper().replace('RATE','').strip() for i in headers]
  headers = [i for i in headers if i]
  df.columns = headers
  df = df[df[headers[0]]!='']
  # Take out years

  years = [i for i in list(df['PERIOD'].values) if '2'==i[0]]
  if len(years)>0:
    years_ind = 0
    period_list = list(df['PERIOD'].values)
    years_ind = [i for i,_ in enumerate(period_list) if period_list[i]==years[0]]
    df = df.iloc[years_ind[0]:]
    df = df[df['PERIOD'].str.slice(0,1)!='2']
    period_list = list(df['PERIOD'].values)
    year_count=0
    period_years=[]
    for k,i in enumerate(period_list):
      period_years.append(i+' '+years[year_count])
      if (k+1)<len(period_list):
        if period_list[k+1]=='January':
          year_count+=1
    df['PERIOD']=period_years
  df = df.replace('',np.nan)
  df = df.dropna(how='any')
  df = df[pd.to_datetime(df['PERIOD'], errors='ignore').notnull()]
  return df

In [11]:
# Get all cpi_inf matches
cpi_inf_combo_list = [k for k, i in enumerate(cpi_inf_page_list) if "cpi_inf_match" in i]

In [12]:
cpi_inf_page_list[129]

{}

In [46]:
df = pd.DataFrame()
for i in cpi_inf_combo_list:

  lei_path = DATA_FOLDER+'/Leading Economic Indicators/'+lei_files[i]
  table_page = str(cpi_inf_page_list[i]['cpi_inf_match'])
  try:
    page_df = get_cpi_inf_data(lei_path, table_page)
    df = pd.concat([df, page_df])

  except:
    print(lei_files[i], i, table_page)


<ipython-input-45-796d45006c48>:36: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = df[pd.to_datetime(df['PERIOD'], errors='ignore').notnull()]
<ipython-input-45-796d45006c48>:36: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = df[pd.to_datetime(df['PERIOD'], errors='ignore').notnull()]
<ipython-input-45-796d45006c48>:36: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = df[pd.to_datetime(df['PERIOD'], errors='ignore').notnull()]
<ipython-input-45-796d45006c48>:36: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `date

In [40]:
table_page = '11'
lei_path = DATA_FOLDER+'/Leading Economic Indicators/LEI April 2008.pdf'
df = camelot.read_pdf(lei_path, pages=table_page, flavor='stream', strip_text=' \n')[0].df
header_catch = 'OVERALL MONTH ON MONTH INFLATION'
header_row = 1
lattice_table = camelot.read_pdf(lei_path, pages=str(table_page), flavor='lattice', strip_text='\n')[0].df
for idx, row in lattice_table.iterrows():
  for i in row:
    print(i)
    if header_catch in i.upper():
      header_row=idx

headers = list(camelot.read_pdf(lei_path, pages=str(table_page), flavor='lattice', strip_text='\n')[0].df.iloc[header_row])
headers = [i.replace('*','').replace('  ',' ').upper().strip() for i in headers]
headers = [i for i in headers if i]

df.columns = headers
df = df[df[headers[0]]!='']
# Take out years

years = [i for i in list(df['PERIOD'].values) if '2'==i[0]]
if len(years)>0:
  years_ind = 0
  period_list = list(df['PERIOD'].values)
  years_ind = [i for i,_ in enumerate(period_list) if period_list[i]==years[0]]
  df = df.iloc[years_ind[0]:]
  df = df[df['PERIOD'].str.slice(0,1)!='2']
  period_list = list(df['PERIOD'].values)
  year_count=0
  period_years=[]
  for k,i in enumerate(period_list):
    period_years.append(i+' '+years[year_count])
    if (k+1)<len(period_list):
      if period_list[k+1]=='January':
        year_count+=1
  df['PERIOD']=period_years
df = df.replace('',np.nan)
df = df.dropna(how='any')
df = df[pd.to_datetime(df['PERIOD'], errors='ignore').notnull()]

PERIOD
NAIROBI INDEX
REST* OF URBAN TOWNS INDEX
OVERALL  KENYA INDEX

OVERALL MONTH ON MONTH INFLATION RATE***

UNDERLYING INFLATION RATE****

2007 January February March April May June July August September October November December 2008 January February March April
225.05 228.18 228.38 222.93 231.39 224.91 226.53 227.64 230.29 230.93 233.73 242.14   266.28 267.04 277.61 282.38
217.85 220.53 223.26 222.39 222.13 224.95 226.60 224.56 226.74 229.37 232.99 238.26   257.39 265.92 274.40 281.55

220.72 223.59 225.30 222.61 221.83 224.93 226.57 225.79 228.16 229.99 233.28 239.81   260.94 266.37 274.55 281.88

9.7 6.8 5.9 5.7 6.3 11.1 13.6 12.4 11.7 10.6 11.8 12.0   18.2 19.1 21.8 26.6

6.4 6.1 6.1 5.9 5.7 5.2 5.6 5.2 5.1 5.2 5.8 5.9   7.6 9.3 9.8 9.6


<ipython-input-40-73b48254e286>:39: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = df[pd.to_datetime(df['PERIOD'], errors='ignore').notnull()]


In [39]:
headers

['PERIOD',
 'NAIROBI INDEX',
 'REST OF URBAN TOWNS INDEX',
 'OVERALL KENYA INDEX',
 '',
 'OVERALL MONTH ON MONTH INFLATION RATE',
 '',
 'UNDERLYING INFLATION RATE',
 '']

In [31]:
period_list

['PERIOD',
 'June',
 'January',
 'February',
 'March',
 'April',
 'May',
 'June',
 'July',
 'August',
 'September',
 'October',
 'November',
 'December',
 'January',
 'February',
 'March']

In [49]:
len(cpi_inf_combo_list)

33

In [50]:
df

,PERIOD,NAIROBI INDEX,REST OF URBAN TOWNS INDEX,OVERALL KENYA INDEX,OVERALL MONTH ON MONTH INFLATION,UNDERLYING INFLATION
6,Feb-08,123.3,120.0,121.3,10.6,6.7
7,Mar-08,125.0,121.1,122.6,11.8,7.1
8,Apr-08,128.6,124.7,126.3,15.9,7.7
9,May-08,131.1,127.3,128.8,18.6,8.7
10,Jun-08,130.2,128.4,129.1,17.8,9.0
...,...,...,...,...,...,...
17,January 2007,225.05,217.85,220.72,9.7,6.4
18,February 2007,228.18,220.53,223.59,6.8,6.1
19,March 2007,228.38,223.26,225.30,5.9,6.1
20,April 2007,222.93,222.39,222.61,5.7,5.9
